# Transformer理解のためのnotebook
transformerを初めて動かして理解したいという方向けのnotebookなので、精度は追求しません。

精度の高いnotebookは, 公開されているnotebookをtransformerで検索すれば見つかるはずです。 

Transformerの説明は[図で理解するTransformer](https://qiita.com/birdwatcher/items/b3e4428f63f708db37b7)で説明してます。
PyTorchの入門は[こちら](https://qiita.com/birdwatcher/items/e8ab9f6bba558759c106)

参考にしたNotebook：
- https://www.kaggle.com/konumaru/saint-with-pytorch-transformer-module
- https://www.kaggle.com/manikanthr5/riiid-sakt-model-training-public

## データ読み込み

In [1]:
import numpy as np
import pandas as pd
from pathlib import Path
import time

In [2]:
input_dir = Path("/kaggle/input/riiid-test-answer-prediction/")

In [3]:
%%time
data = pd.read_csv(input_dir / "train.csv",
                   usecols=[1, 2, 3, 4, 7],
                   dtype={
                        'timestamp': 'int64',
                        'user_id': 'int32',
                        'content_id': 'int16',
                        'content_type_id': 'int8',
                        'answered_correctly':'int8',
                   }
       )
# 2分くらいかかる

CPU times: user 57.3 s, sys: 3.39 s, total: 1min
Wall time: 2min 15s


In [4]:
# 問題のマスタ
questions = pd.read_csv(input_dir/"questions.csv",
                        dtype={
                            "question_id":np.int16,
                            "bundle_id":np.int16,
                            "correct_answer":np.int8,
                            "part":np.int8,
                        }
           )
n_questions = len(questions)
n_questions

13523

In [5]:
del questions

In [6]:
data.head()

,timestamp,user_id,content_id,content_type_id,answered_correctly
0,0,115,5692,0,1
1,56943,115,5716,0,1
2,118363,115,128,0,1
3,131167,115,7860,0,1
4,137965,115,7922,0,1


### データ概要
- 問題に正解したかどうか answerd_correctly $\in \{0,1\}$ を予測するタスク
- content_id: 問題のID/講義のID
- content_type_id: 問題=0, 講義=1

In [7]:
# 講義のデータを落とす & 時系列順にソート
data = data[data["content_type_id"] == 0].sort_values('timestamp').reset_index(drop = True)

In [8]:
data.shape

(99271300, 5)

In [9]:
# メモリが貧弱なのでデータを絞る
data=data.tail(10000000)

In [10]:
%%time
# ユーザーごとに「問題IDの系列」と「正解不正解の系列」を持つ形に変形
data = data.groupby("user_id").apply(
    lambda row: (
        row["content_id"].values,
        row["answered_correctly"].values,
    )
)
data

CPU times: user 2.58 s, sys: 113 ms, total: 2.69 s
Wall time: 3.21 s


user_id
138650        ([6430, 4931, 6670, 9113, 4079, 5000, 3609, 88...
238966        ([400, 10685, 737, 1078, 457, 10686, 962, 522,...
286187                   ([4108, 876, 14, 10525], [0, 0, 1, 0])
359283        ([244, 12214, 1326, 307, 520, 11999, 422, 902,...
382571        ([984, 205, 468, 12132, 801, 889, 294, 12160, ...
                                    ...                        
2147190658    ([6911, 6909, 6910, 6908, 3584, 7951, 9102, 11...
2147192453    ([4478, 5956, 4966, 8430, 4480, 6675, 4188, 86...
2147331033    ([5161, 8635, 4402, 13267, 9034, 11232, 6475, ...
2147382115    ([1291, 426, 335, 10684, 580, 1335, 611, 10685...
2147413636    ([828, 11927, 784, 390, 1149, 11940, 2268, 226...
Length: 27789, dtype: object

In [11]:
# モデルで扱う最大系列長
MAX_SEQ = 20
# モデルで考慮する最小サンプル数（このサンプル数未満のユーザーは無視される）
MIN_SAMPLES = 5
# 埋め込み次元数
EMBED_DIM = 32
# Attentionヘッドの数
NUM_HEADS = 2
# ドロップアウト割合
DROPOUT_RATE = 0.2
# 学習率
LEARNING_RATE = 1e-3
# 最大学習率
MAX_LEARNING_RATE = 2e-3
# エポック数（学習データ全体を何周するか）
EPOCHS = 5
# バッチサイズ
BATCH_SIZE = 1024

In [12]:
# 学習に使うユーザー数
TRAIN_SAMPLES = int(data.shape[0]*0.8)
# 学習データと検証データに分ける
train_index = data.index.to_list()[:TRAIN_SAMPLES]
valid_index = data.index.to_list()[TRAIN_SAMPLES:]
train = data[data.index.isin(train_index)]
valid = data[data.index.isin(valid_index)]
print(len(train), len(valid))

22231 5558


In [13]:
del data, train_index, valid_index

## モデル作成

In [14]:
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
from sklearn.metrics import roc_auc_score
from tqdm.notebook import tqdm

### 普通のTransfomer
- いよいよ本題
- まずは普通に思いつきそうなTransformerから

![](https://camo.qiitausercontent.com/4d79e3addad65aa484d0cf9625de272187dc3b39/68747470733a2f2f71696974612d696d6167652d73746f72652e73332e61702d6e6f727468656173742d312e616d617a6f6e6177732e636f6d2f302f3239343636312f32373266663064382d353537352d333462372d366339372d3138363131386631343935382e706e67)

#### データローダー
作るモデルに合ったデータセットの形式にする

In [15]:
# 系列の開始記号（Begin Of Sequence）
BOS = 2
class TransformerDataset(Dataset):
    """普通の場合"""
    def __init__(self, group, n_questions, min_samples=1, max_seq=128):
        super(TransformerDataset, self).__init__()
        self.max_seq = max_seq
        self.n_questions = n_questions
        # ユーザーID→系列を格納する変数
        self.samples = {}
        
        self.user_ids = []
        for user_id in group.index:
            q, qa = group[user_id]
            # サンプルが少ないユーザーは無視
            if len(q) < min_samples:
                continue
            
            # 最大系列長より長い系列の場合
            if len(q) > self.max_seq:
                total_questions = len(q)
                # 最初の端数分の系列を格納
                initial = total_questions % self.max_seq
                if initial >= min_samples:
                    self.user_ids.append(f"{user_id}_0")
                    self.samples[f"{user_id}_0"] = (q[:initial], qa[:initial])
                # 残りの長い系列について最大系列長ずつ取り出して格納
                for seq in range(total_questions // self.max_seq):
                    self.user_ids.append(f"{user_id}_{seq+1}")
                    start = initial + seq * self.max_seq
                    end = start + self.max_seq
                    self.samples[f"{user_id}_{seq+1}"] = (q[start:end], qa[start:end])
            else:
                # 最大系列長より短い系列の場合
                user_id = str(user_id)
                self.user_ids.append(user_id)
                self.samples[user_id] = (q, qa)
    
    def __len__(self):
        return len(self.user_ids)

    def __getitem__(self, index):
        user_id = self.user_ids[index]
        q_, qa_ = self.samples[user_id]
        seq_len = len(q_)
        # 最大系列長で揃える
        q = np.zeros(self.max_seq, dtype=int)
        qa = np.zeros(self.max_seq, dtype=int)
        qa_shift = np.zeros(self.max_seq, dtype=int)
        if seq_len == self.max_seq:
            q[:] = q_
            qa[:] = qa_
        else:# 最大長ないものは末尾に格納
            q[-seq_len:] = q_
            qa[-seq_len:] = qa_
        # 右シフトしたもの (Decoderの入力)
        qa_shift[-seq_len:] = np.concatenate([[BOS],qa_[:-1]])
        
        # transformerで無視する部分を指定するマスク
        # True: マスクされる、False: マスクなし
        padding_mask = np.ones(self.max_seq, dtype=bool)
        # 末尾のデータ格納箇所はマスクしない
        padding_mask[-seq_len:] = False
        
        return q, qa_shift, qa, padding_mask

In [16]:
train_dataset = TransformerDataset(train, n_questions, min_samples=MIN_SAMPLES, max_seq=MAX_SEQ)
train_dataloader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True, num_workers=3)
valid_dataset = TransformerDataset(valid, n_questions, max_seq=MAX_SEQ)
valid_dataloader = DataLoader(valid_dataset, batch_size=BATCH_SIZE, shuffle=False, num_workers=3)

#### Transformer

In [17]:
class FFN(nn.Module):
    """Feed Foward Network"""
    def __init__(self, state_size=200):
        super(FFN, self).__init__()
        self.state_size = state_size
        # 線形変換とReLU
        self.lr1 = nn.Linear(state_size, state_size)
        self.relu = nn.ReLU()
        self.lr2 = nn.Linear(state_size, state_size)
        self.dropout = nn.Dropout(0.2)
    
    def forward(self, x):
        x = self.lr1(x)
        x = self.relu(x)
        x = self.lr2(x)
        return self.dropout(x)

def future_mask(seq_length):
    """未来情報を見ないためのマスク"""
    # 対角が0で右上の三角部分が1の上三角行列
    future_mask = np.triu(np.ones((seq_length, seq_length)), k=1).astype('bool')
    return torch.from_numpy(future_mask)

class Transformer(nn.Module):
    def __init__(self, n_questions, n_response, max_seq=128, embed_dim=128, num_heads=8, dropout_rate=0.2):
        super(Transformer, self).__init__()
        self.n_questions = n_questions # 問題の数
        self.n_response = n_response # {0, 1, BOS}の3種類
        self.embed_dim = embed_dim
        self.max_seq = max_seq
        # Embedding系
        # 離散表現（IDなどの整数）を指定した次元数の分散表現に変換する層（自然言語でいうところのword2vec）
        # nn.Embedding(単語の種類数, 埋め込みたい次元数)
        # nn.Embedding内でやっていることは単純で、onehotベクトルを作ってから線形変換しているだけっぽい
        self.position_embed_e = nn.Embedding(max_seq, embed_dim)
        self.question_embed_e = nn.Embedding(n_questions, embed_dim)
        self.position_embed_d = nn.Embedding(max_seq, embed_dim)
        self.response_embed_d = nn.Embedding(n_response, embed_dim)
        # Attention系
        # 直前のLinearはこの中に含まれている
        # See https://pytorch.org/docs/stable/generated/torch.nn.MultiheadAttention.html
        self.attention_e = nn.MultiheadAttention(embed_dim=embed_dim, num_heads=num_heads, dropout=dropout_rate)
        self.attention_d = nn.MultiheadAttention(embed_dim=embed_dim, num_heads=num_heads, dropout=dropout_rate)
        self.attention_ed = nn.MultiheadAttention(embed_dim=embed_dim, num_heads=num_heads, dropout=dropout_rate)
        # 正規化系
        self.layer_normal1_e = nn.LayerNorm(embed_dim)
        self.layer_normal2_e = nn.LayerNorm(embed_dim)
        self.layer_normal1_d = nn.LayerNorm(embed_dim)
        self.layer_normal2_d = nn.LayerNorm(embed_dim)
        self.layer_normal3_d = nn.LayerNorm(embed_dim)
        # FFN系
        self.ffn_e = FFN(embed_dim)
        self.ffn_d = FFN(embed_dim)
        # 最後の線形変換
        self.pred = nn.Linear(embed_dim, 1)
    
    def forward(self, question_ids, responses):
        device = question_ids.device
        #######################################################################
        # Encoder
        #######################################################################
        # --------- Embedding ---------
        # 系列の何番目かを表す[[0, 1, 2, ..., seq_len - 1]]を生成
        seq = torch.arange(self.max_seq, device=device).unsqueeze(0)
        # それをEmbedding（分散表現へ）
        pos_e = self.position_embed_e(seq)
        # Encoderに入れるID列を分散表現へ
        question = self.question_embed_e(question_ids)
        # 位置情報を加える
        enc = pos_e + question
        # --------- Attention ---------
        # Attentionの関数は、（系列長, バッチサイズ, 次元数）の形で受け取る
        enc = enc.permute(1, 0, 2) # （バッチサイズ, 系列長, 次元数）=>（系列長, バッチサイズ, 次元数）
        # self-attention
        enc_tmp, _ = self.attention_e(enc, enc, enc)
        enc = self.layer_normal1_e(enc_tmp+enc)
        # --------- Feed Forward Network ---------
        enc_tmp = self.ffn_e(enc)
        enc = self.layer_normal2_e(enc_tmp+enc)
        #######################################################################
        # Decoder
        #######################################################################
        # --------- Embedding ---------
        # 位置情報をEmbedding（分散表現へ）
        pos_d = self.position_embed_d(seq)
        # Decoderに入れる離散表現を分散表現へ
        response = self.response_embed_d(responses)
        dec = response + pos_d
        # --------- Attention ---------
        # 未来情報を隠すマスク
        att_mask = future_mask(self.max_seq).to(device)
        # Attentionの関数は、（系列長, バッチサイズ, 次元数）の形で受け取る
        dec = dec.permute(1, 0, 2) # （バッチサイズ, 系列長, 次元数）=>（系列長, バッチサイズ, 次元数）
        # self-attention
        dec_tmp, _ = self.attention_d(dec, dec, dec, attn_mask=att_mask)
        dec = self.layer_normal1_d(dec_tmp+dec)
        # source-target attention
        dec_tmp, _ = self.attention_ed(dec, enc, enc)
        dec = self.layer_normal2_d(dec_tmp+dec)
        # --------- Feed Forward Network ---------
        dec_tmp = self.ffn_d(dec)
        dec = self.layer_normal3_d(dec_tmp+dec)
        # shapeをもとに戻す
        dec = dec.permute(1, 0, 2) # （系列長, バッチサイズ, 次元数）=>（バッチサイズ, 系列長, 次元数）

        # 最後の線形変換
        dec = self.pred(dec)

        return dec.squeeze(-1)

- 今回は[nn.MultiheadAttention](https://pytorch.org/docs/stable/generated/torch.nn.MultiheadAttention.html)を使って他はすべて自分で書いた
- AttentionからFeed Forward Networkまでの一連の内容をサボりたければ[nn.Transformer](https://pytorch.org/docs/stable/generated/torch.nn.Transformer.html)モジュールを使う方法もある
    - [実装を確認](https://pytorch.org/docs/stable/_modules/torch/nn/modules/transformer.html#Transformer)すると、Attention, FFN, 正規化までやっているようだ
    - これを使えば、自分で書くべき箇所はEmbeddingと最後の線形変換だけになる
- Attentionの実装が気になる場合は[こちら](https://github.com/jadore801120/attention-is-all-you-need-pytorch/tree/132907dd272e2cc92e3c10e6c4e783a87ff8893d)の実装がわかりやすいと感じた
    - ライブラリにあるnn.MultiheadAttentionの実装ではなく、独自でMultiHeadAttentionを書いている

#### 最適化法と損失の設定

In [18]:
# 0: 不正解、1：正解、2：BOS
n_responses = 3

# モデルの設定
device = "cuda" if torch.cuda.is_available() else "cpu"
model = Transformer(n_questions, n_responses,
                    max_seq=MAX_SEQ,
                    embed_dim=EMBED_DIM,
                    num_heads=NUM_HEADS,
                   )
# 最適化法の指定
optimizer = torch.optim.Adam(model.parameters(), lr=LEARNING_RATE)
# 損失の設定
# - BCEWithLogitsLoss: sigmoidをとる前の値を入力として受け付ける
# - BCELoss: sigmoidをとった後の値を入力として受け付ける
# モデル自体にsigmoidを含めてないため、前者を使う（この方が数値的に安定していて、高速らしい）
criterion = nn.BCEWithLogitsLoss()
# 学習率をどう動かしていくか
scheduler = torch.optim.lr_scheduler.OneCycleLR(
    optimizer, max_lr=MAX_LEARNING_RATE, steps_per_epoch=len(train_dataloader), epochs=EPOCHS
)

model.to(device)
criterion.to(device)

BCEWithLogitsLoss()

#### 学習と評価のループ

In [19]:
def train_fn(model, dataloader, optimizer, scheduler, criterion, device="cpu"):
    """学習ループ"""
    model.train()

    train_loss = []
    num_corrects = 0
    num_total = 0
    targets = []
    outs = []

    for item in tqdm(dataloader):
        # tensorを置くデバイスの設定
        enc_input = item[0].to(device).long()
        dec_input = item[1].to(device).long()
        target = item[2].to(device).float()
        padding_mask = item[3].to(device).bool()
        # 勾配情報の初期化
        optimizer.zero_grad()
        # モデルの出力計算
        output = model(enc_input, dec_input)
        # 予測値のうち、maskされていない場所だけ取り出す
        output = torch.masked_select(output, torch.logical_not(padding_mask))
        # 真値のうち、maskされていない場所だけ取り出す
        target = torch.masked_select(target, torch.logical_not(padding_mask))
        # 損失を計算
        loss = criterion(output, target)
        # 勾配を計算
        loss.backward()
        # 最適化法に基づいてパラメータ更新
        optimizer.step()
        scheduler.step()
        # 損失を記録
        train_loss.append(loss.item())
        # 0.5以上なら1と予測
        pred = (torch.sigmoid(output) >= 0.5).long()
        # 精度のための記録
        num_corrects += (pred == target).sum().item()
        num_total += len(target)

        targets.extend(target.view(-1).data.cpu().numpy())
        outs.extend(output.view(-1).data.cpu().numpy())

    acc = num_corrects / num_total
    auc = roc_auc_score(targets, outs)
    loss = np.mean(train_loss)

    return loss, acc, auc

In [20]:
def valid_fn(model, dataloader, criterion, device="cpu"):
    """検証のループ"""
    model.eval()

    valid_loss = []
    num_corrects = 0
    num_total = 0
    targets = []
    outs = []

    for item in tqdm(dataloader):
        enc_input = item[0].to(device).long()
        dec_input = item[1].to(device).long()
        target = item[2].to(device).float()
        padding_mask = item[3].to(device).bool()
        # モデルの出力計算
        output = model(enc_input, dec_input)
        # 予測値のうち、maskされていない場所だけ取り出す
        output = torch.masked_select(output, torch.logical_not(padding_mask))
        # 真値のうち、maskされていない場所だけ取り出す
        target = torch.masked_select(target, torch.logical_not(padding_mask))
        # 損失を計算
        loss = criterion(output, target)
        # 損失を記録
        valid_loss.append(loss.item())
        # 0.5以上なら1と予測
        pred = (torch.sigmoid(output) >= 0.5).long()
        # 精度のための記録
        num_corrects += (pred == target).sum().item()
        num_total += len(target)

        targets.extend(target.view(-1).data.cpu().numpy())
        outs.extend(output.view(-1).data.cpu().numpy())

    acc = num_corrects / num_total
    auc = roc_auc_score(targets, outs)
    loss = np.mean(valid_loss)

    return loss, acc, auc

In [21]:
best_auc = 0
early_stop = 3
step = 0
for epoch in range(EPOCHS):
    loss, acc, auc = train_fn(model, train_dataloader, optimizer, scheduler, criterion, device)
    print(f"[train] epoch: {epoch+1}/{EPOCHS}, loss: {loss:.3f}, acc: {acc:.3f}, auc: {auc:.3f}")
    loss, acc, auc = valid_fn(model, valid_dataloader, criterion, device)
    print(f"[valid] epoch: {epoch+1}/{EPOCHS}, loss: {loss:.3f}, acc: {acc:.3f}, auc: {auc:.3f}")
    if auc > best_auc:
        best_auc = auc
        step = 0
        torch.save(model.state_dict(), "model.pt")
    else:
        step += 1
        if step >= early_stop:
            break

  0%|          | 0/397 [00:00<?, ?it/s]

[train] epoch: 1/5, loss: 0.622, acc: 0.674, auc: 0.574


  0%|          | 0/100 [00:00<?, ?it/s]

[valid] epoch: 1/5, loss: 0.612, acc: 0.680, auc: 0.615


  0%|          | 0/397 [00:00<?, ?it/s]

...

[valid] epoch: 4/5, loss: 0.549, acc: 0.719, auc: 0.740


  0%|          | 0/397 [00:00<?, ?it/s]

[train] epoch: 5/5, loss: 0.547, acc: 0.722, auc: 0.736


  0%|          | 0/100 [00:00<?, ?it/s]

[valid] epoch: 5/5, loss: 0.547, acc: 0.720, auc: 0.741


### SAKT model
- [A Self-Attentive model for Knowledge Tracing](https://arxiv.org/pdf/1907.06837.pdf)というものがあるらしい
- 問題IDと正解不正解の系列をペアでEncoderに入力するモデル

![](https://camo.qiitausercontent.com/efc4eefce7364cf4cb3c3d3dcc13472e61c59a81/68747470733a2f2f71696974612d696d6167652d73746f72652e73332e61702d6e6f727468656173742d312e616d617a6f6e6177732e636f6d2f302f3239343636312f32653961363366642d643231622d646366622d306134612d6262613038356634616661372e706e67)

#### データローダー
作るモデルに合ったデータセットの形式にする

In [22]:
class SAKTDataset(Dataset):
    def __init__(self, group, n_questions, min_samples=1, max_seq=128):
        super(SAKTDataset, self).__init__()
        self.max_seq = max_seq
        self.n_questions = n_questions
        # ユーザーID→系列を格納する変数
        self.samples = {}
        
        self.user_ids = []
        for user_id in group.index:
            q, qa = group[user_id]
            # サンプルが少ないユーザーは無視
            if len(q) < min_samples:
                continue
            
            # 最大系列長より長い系列の場合
            if len(q) > self.max_seq:
                total_questions = len(q)
                # 最初の端数分の系列を格納
                initial = total_questions % self.max_seq
                if initial >= min_samples:
                    self.user_ids.append(f"{user_id}_0")
                    self.samples[f"{user_id}_0"] = (q[:initial], qa[:initial])
                # 残りの長い系列について最大系列長ずつ取り出して格納
                for seq in range(total_questions // self.max_seq):
                    self.user_ids.append(f"{user_id}_{seq+1}")
                    start = initial + seq * self.max_seq
                    end = start + self.max_seq
                    self.samples[f"{user_id}_{seq+1}"] = (q[start:end], qa[start:end])
            else:
                # 最大系列長より短い系列の場合
                user_id = str(user_id)
                self.user_ids.append(user_id)
                self.samples[user_id] = (q, qa)
    
    def __len__(self):
        return len(self.user_ids)

    def __getitem__(self, index):
        user_id = self.user_ids[index]
        q_, qa_ = self.samples[user_id]
        seq_len = len(q_)
        # 最大系列長で揃える
        q = np.zeros(self.max_seq, dtype=int)
        qa = np.zeros(self.max_seq, dtype=int)
        if seq_len == self.max_seq:
            q[:] = q_
            qa[:] = qa_
        else:
            q[-seq_len:] = q_
            qa[-seq_len:] = qa_
        # 右シフトしたもの
        # 問題のID系列 (decoderの入力)
        target_id = q[1:]
        # 正解不正解の系列 (予測したいもの)
        label = qa[1:]

        # 正解：コンテンツID + 全質問数
        # 不正解：コンテンツID
        # 数字で列挙すると、不正解が並んだあとに、正解が並ぶイメージ
        # そのような数字が並んだ系列をEncoderの入力にする
        # 最後の要素は予測対象なため除外
        x = q[:-1] + (qa[:-1] == 1) * self.n_questions

        # 無視する部分を指定するマスク
        # True: マスクされる、False: マスクなし
        padding_mask = np.ones(self.max_seq, dtype=bool)
        # 末尾のデータ格納箇所はマスクしない
        padding_mask[-seq_len:] = False
        padding_mask = padding_mask[1:]

        return x, target_id, label, padding_mask

In [23]:
train_dataset = SAKTDataset(train, n_questions, min_samples=MIN_SAMPLES, max_seq=MAX_SEQ)
train_dataloader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True, num_workers=3)
valid_dataset = SAKTDataset(valid, n_questions, max_seq=MAX_SEQ)
valid_dataloader = DataLoader(valid_dataset, batch_size=BATCH_SIZE, shuffle=False, num_workers=3)

#### SAKT

In [24]:
class SAKT(nn.Module):
    """SAKTモデル"""
    def __init__(self, n_questions, max_seq=128, embed_dim=128, num_heads=8, dropout_rate=0.2):
        super(SAKT, self).__init__()
        self.n_questions = n_questions
        self.embed_dim = embed_dim

        self.embedding = nn.Embedding(2*n_questions, embed_dim)
        self.pos_embedding = nn.Embedding(max_seq-1, embed_dim)
        self.e_embedding = nn.Embedding(n_questions, embed_dim)
        # MultiheadAttentionというクラスが用意されている
        self.multi_att = nn.MultiheadAttention(embed_dim=embed_dim, num_heads=num_heads, dropout=dropout_rate)

        self.dropout = nn.Dropout(dropout_rate)
        self.layer_normal = nn.LayerNorm(embed_dim) 

        self.ffn = FFN(embed_dim)
        self.pred = nn.Linear(embed_dim, 1)
    
    def forward(self, x, question_ids):
        device = x.device
        # ID系列の埋め込み
        x = self.embedding(x)
        # 系列の何番目かを表す[[0, 1, 2, ..., seq_len - 1]]を生成
        pos_id = torch.arange(x.size(1)).unsqueeze(0).to(device)
        # 系列内位置情報の埋め込み
        pos_x = self.pos_embedding(pos_id)

        x = x + pos_x
        # 問題の系列も埋め込み
        e = self.e_embedding(question_ids)

        x = x.permute(1, 0, 2) # x: [bs, s_len, embed] => [s_len, bs, embed]
        e = e.permute(1, 0, 2)
        # 未来情報を隠すマスク
        att_mask = future_mask(x.size(0)).to(device)
        att_output, att_weight = self.multi_att(e, x, x, attn_mask=att_mask)
        att_output = self.layer_normal(att_output + e)
        att_output = att_output.permute(1, 0, 2) # att_output: [s_len, bs, embed] => [bs, s_len, embed]

        x = self.ffn(att_output)
        x = self.layer_normal(x + att_output)
        x = self.pred(x)

        return x.squeeze(-1)

- `FNN`と`future_mask`は、普通のTransformerの方で定義したものを使ってます

#### 最適化法と損失の設定

In [25]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

model = SAKT(n_questions, max_seq=MAX_SEQ, embed_dim=EMBED_DIM, num_heads=NUM_HEADS, dropout_rate=DROPOUT_RATE)
optimizer = torch.optim.Adam(model.parameters(), lr=LEARNING_RATE)
criterion = nn.BCEWithLogitsLoss()
scheduler = torch.optim.lr_scheduler.OneCycleLR(
    optimizer, max_lr=MAX_LEARNING_RATE, steps_per_epoch=len(train_dataloader), epochs=EPOCHS
)

model.to(device)
criterion.to(device)

BCEWithLogitsLoss()

#### 学習
学習のループは普通のTransformerの方の関数と同じものを使います。

In [26]:
best_auc = 0
early_stop = 3
step = 0
for epoch in range(EPOCHS):
    loss, acc, auc = train_fn(model, train_dataloader, optimizer, scheduler, criterion, device)
    print(f"[train] epoch: {epoch+1}/{EPOCHS}, loss: {loss:.3f}, acc: {acc:.3f}, auc: {auc:.3f}")
    loss, acc, auc = valid_fn(model, valid_dataloader, criterion, device)
    print(f"[valid] epoch: {epoch+1}/{EPOCHS}, loss: {loss:.3f}, acc: {acc:.3f}, auc: {auc:.3f}")
    if auc > best_auc:
        best_auc = auc
        step = 0
        torch.save(model.state_dict(), "sakt_model.pt")
    else:
        step += 1
        if step >= early_stop:
            break

  0%|          | 0/397 [00:00<?, ?it/s]

[train] epoch: 1/5, loss: 0.643, acc: 0.654, auc: 0.521


  0%|          | 0/100 [00:00<?, ?it/s]

[valid] epoch: 1/5, loss: 0.621, acc: 0.675, auc: 0.586


  0%|          | 0/397 [00:00<?, ?it/s]

...

[valid] epoch: 4/5, loss: 0.549, acc: 0.720, auc: 0.738


  0%|          | 0/397 [00:00<?, ?it/s]

[train] epoch: 5/5, loss: 0.544, acc: 0.724, auc: 0.740


  0%|          | 0/100 [00:00<?, ?it/s]

[valid] epoch: 5/5, loss: 0.549, acc: 0.720, auc: 0.738
